In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

# Exploring the mathematics of Guild Ball

Hello! This is a Jupyter notebook, basically a self-contained playground for Python programming. Below you'll find some interactive visualisations of the probabilities involved in the miniatures game [Guild Ball](http://steamforged.com/guildball) (along with the code that makes them, if you're into that). To play around with the visualisations, simply select `Cell>Run All` from the menus above.

## Target Number Tests
### Probabilities & expected values
Below is a visualisation of the probabilities involved in a general TN test. It also shows the expected value of the test, which is the average number of hits. You can use the sliders to adjust the dice pool and target number for the test.

In [97]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Scatter, Layout, Data
import plotly.figure_factory as ff
from scipy.stats import binom
from ipywidgets import interact, IntSlider

init_notebook_mode(connected=True)


def tn_test(dice, tn, cumm=True):
    '''
    Returns a dictionary representing the probability distribution
    for a TN test with target number `tn` and dice pool `dice`. Each
    key is a number of hits. If `cumm` is True, the corresponding value
    is the probability of getting at least that many hits. Otherwise it
    is the probability of getting exactly that many hits.
    '''
    hit_range = list(range(dice+1))
    if cumm:
        return {h: round(binom.cdf(dice-h, dice, (tn-1)/6), 3) for h in hit_range}
    return {h: round(binom.pmf(h, dice, (7-tn)/6), 3) for h in hit_range}

dice_slider = IntSlider(min=1, max=12, description="Dice Pool")
tn_slider = IntSlider(min=2, max=6, description="Target Number")
@interact(dice=dice_slider, tn=tn_slider)
def plot_succeses(dice, tn):
    # Data generation
    hit_range, probabilities = zip(*tn_test(dice, tn).items())
    percentages = list(map(lambda p: p * 100, probabilities))

    # Table
    table_data = [['Number of hits', 'Probability (%)'],
                  *(['{}+'.format(n), percentages[n]] for n in hit_range)
                 ]
    probs_figure = ff.create_table(table_data, height_constant=80)

    # Scatter plot
    trace = Scatter(x=hit_range, y=percentages, mode='markers', xaxis='x2', yaxis='y2')
    probs_figure['data'].extend(Data([trace]))

    probs_figure.layout.xaxis.update({'domain': [0, .33]})
    probs_figure.layout.xaxis2.update({'domain': [0.43, 1.],
                                       'title': 'Number of hits',
                                       'ticktext': ['{}+'.format(n) for n in hit_range],
                                       'tickvals': hit_range
                                })
    probs_figure.layout.yaxis2.update({'anchor': 'x2',
                                       'title': 'Probability (%)',
                                       'range': (0, 110)
                                })
    probs_figure.layout.margin.update({'t':50, 'b':100})
    probs_figure.layout.update({'title': 'Probability of N successes for a given TN and dice pool'})
    
    iplot(probs_figure)
    # Expected value
    table = ff.create_table([['Expected value:', round(dice*(7-tn)/6, 1)]], font_colors=['#000000'])
    table.layout.margin.update({'l': 300, 'r': 300})
    iplot(table)

### Confidence in hits
The probabilities can be recast in a more useful way. Below is a table showing the number of hits that can be expected with a certain confidence (say, 80%). In other words, it shows you how many dice you have to roll to be confident of a certain number of hits. You can adjust the target number for the test and required confidence level using the sliders.

In [78]:
from ipywidgets import FloatSlider
confidence_slider = FloatSlider(min=0,
                                max=1,
                                step=0.01,
                                value=0.75,
                                description='Confidence level',
                                readout_format='.0%',
                               )
@interact(confidence=confidence_slider, tn=tn_slider)
def confidence_table(confidence, tn):
    table_data = [['Dice pool', 'Maximum number of hits with {}% confidence'.format(confidence*100)]]
    for dice in range(1, 13):    # Show results for up to 12 dice
        best_result = max(map(lambda t: t[0], filter(lambda t: t[1] >= confidence, tn_test(dice, tn).items())))
        table_data.append([dice, best_result])
    iplot(ff.create_table(table_data))